In [1]:
# Paramètres : datedebut / datefin / typepublication / sourcecouplage 
# / table_transaction et ses colonnes / colonne de jointure date / colonne de jointure annonce
# / table_bien et ses colonnes / colonne de jointure date / colonne de jointure annonce
# 

from sql_template_instant_t import query

def format_query(date_t, 
                typepublication,
                sourcecouplage,
                dim_annonce_col_list,
                annonce_col_list,
                typetransaction_table,
                typetransaction_col_list,
                typebien_table,
                typebien_col_list, 
                query
                ):

    # Adding table alias for each col in list (list)
    dim_annonce_col_list_with_alias = ['t1.' + col for col in dim_annonce_col_list]
    dim_annonce_col_list_with_alias2 = ['full_dwh.' + col for col in dim_annonce_col_list]
    annonce_col_list_with_alias = ['t2.' + col for col in annonce_col_list]
    typebien_col_list_with_alias = ['t3.' + col for col in typebien_col_list]
    typetransaction_col_list_with_alias = ['t4.' + col for col in typetransaction_col_list]

    # joining them together (str)
    dim_annonce_col_string = ', '.join(dim_annonce_col_list_with_alias)
    dim_annonce_col_string2 = ', '.join(dim_annonce_col_list_with_alias2)
    annonce_col_string = ', '.join(annonce_col_list_with_alias)
    typebien_col_string = ', '.join(typebien_col_list_with_alias)
    typetransaction_col_string = ', '.join(typetransaction_col_list_with_alias)

    # Full columns for all the tables (str)
    full_cols = dim_annonce_col_string + ', ' + dim_annonce_col_string2 + ', ' + \ 
    annonce_col_string + ', ' + typebien_col_string + ', ' + typetransaction_col_string

    # Full columns for dim_annonce (str)
    dim_annonce_cols = ', '.join(dim_annonce_col_list)

    # Full columns for annonce_history (str)
    annonce_cols = ', '.join(annonce_col_list)

    # Full columns for typetransaction_table (str)
    transaction_cols = ', '.join(typetransaction_col_list)

    # Full columns for typebien_table (str)
    bien_cols = ', '.join(typebien_col_list)
    
    # Join cols
    idannonce_join_for_annonce = annonce_col_list[0]
    idannonce_join_for_typetransaction = typetransaction_col_list[0]
    idannonce_join_for_typebien = typebien_col_list[0]
    
    # Config_dict
    config_dict = {'full_cols': full_cols,
                   'date_t': date_t,
                   'typepublication': typepublication, 
                   'sourcecouplage': sourcecouplage,
                   'dim_annonce_cols': dim_annonce_cols,
                   'annonce_cols': annonce_cols,
                   'transaction_cols': transaction_cols,
                   'typebien_table': typebien_table,
                   'typetransaction_table': typetransaction_table,
                   'bien_cols': bien_cols,
                   'idannonce_join_for_annonce': idannonce_join_for_annonce,
                   'idannonce_join_for_typetransaction': idannonce_join_for_typetransaction,
                   'idannonce_join_for_typebien': idannonce_join_for_typebien
                  }
    
    query = query.format(**config_dict)
    
    return(query)


In [2]:
from sql_template_instant_t import query

query = format_query(date_t = '2020-03-01',
                    typepublication = 1,
                    sourcecouplage = 0,
                    dim_annonce_col_list = ['iddwh_annonce'],
                    annonce_col_list = ['idannonce', 'ville', 'cp', 
                                        'descriptif_fr', 'libelle_fr', 
                                        'latitude', 'longitude', 
                                        'idtypebien', 'idtypetransaction', 
                                        'idquartier', 'idiris', 
                                        'dt_crea', 'dt_maj', 
                                        'codeinsee', 'idagence'],
                    typetransaction_table = 'immobc.atvente_history',
                    typetransaction_col_list = ['idannonce', 'px', 'prixmaxi'],
                    typebien_table = 'immobc.abmaison_history',
                    typebien_col_list = ['idannonce', 'si_meuble'], 
                    query = query
                    )

In [3]:
print(query)


SELECT t1.iddwh_annonce, t2.idannonce, t2.ville, t2.cp, t2.descriptif_fr, t2.libelle_fr, t2.latitude, t2.longitude, t2.idtypebien, t2.idtypetransaction, t2.idquartier, t2.idiris, t2.dt_crea, t2.dt_maj, t2.codeinsee, t2.idagence, t3.idannonce, t3.si_meuble, t4.idannonce, t4.px, t4.prixmaxi FROM


      (
           SELECT iddwh_annonce
                  from dwhdweb.dimannoncepublicationhisto
                  WHERE (datedebut >=   TIMESTAMP '2020-03-01' AND
                        (iddwh_typepublication = 1 AND iddwh_typepublicationcouplage = 0))
      )
      AS t1


LEFT JOIN


     (
           SELECT *
                      FROM
                         (
                          SELECT
                               idannonce, ville, cp, descriptif_fr, libelle_fr, latitude, longitude, idtypebien, idtypetransaction, idquartier, idiris, dt_crea, dt_maj, codeinsee, idagence,
                               CAST(from_iso8601_timestamp(CONCAT(CAST(year AS VARCHAR),'-',LPAD(CAST(month 